In [ ]:
f = open("test.en","r")
f = f.readlines()
english_sentences = []
for line in f:
  ws = np.array(re.split(" |\n",line)[:-1])
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in ws]
  
  try:
      stripped.remove('')
  except ValueError:
      pass 
  english_sentences.append(stripped)
 
english_sentences = np.array(english_sentences)
 
f = open("test.ta","r")
f = f.readlines()
tamil_sentences = []
for line in f:
  ws = np.array(re.split(" |\n",line)[:-1])
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in ws]
  
  try:
      stripped.remove('')
  except ValueError:
      pass 
  tamil_sentences.append(stripped)
 
tamil_sentences = np.array(tamil_sentences)
 
np.save("english_sentences_without_punctuation_test.npy",english_sentences)
np.save("tamil_sentences_without_punctuation_test.npy",tamil_sentences)

In [ ]:
# Extracting unique words : vocabulary
english_sentences = np.load("english_sentences_without_punctuation_train.npy",allow_pickle=True)
tamil_sentences = np.load("tamil_sentences_without_punctuation_train.npy",allow_pickle=True)
def get_vocab(sentences):
  words = []
  for sentence in sentences:
    for word in sentence:
      words.append(word.lower())
  words.append("start_token")
  words.append("end_token")
  words = np.array(words).flatten()
  words = np.unique(words)
  return words

english_words = get_vocab(english_sentences)
tamil_words = get_vocab(tamil_sentences)

np.save("english_words_without.npy",english_words)
np.save("tamil_words_without.npy",tamil_words)
print(tamil_words)
print(tamil_words.shape)
print(english_words.shape)


['' '00' '00மணிக்கு♪' ... '♪வேண்டும்குழந்தைசுற்றிகுழப்பம்' '♪ஷைன்பிரித்து'
 '♪♪']
(18750,)
(9983,)


In [ ]:
# Creating initial embeddings for all english words in the vocabulary 
 
input_file = open('../6B.300_embeddings.pkl', "rb")
glove = pickle.load(input_file)
idx_file = open('../6B.300_idx.pkl', "rb")
idx = pickle.load(idx_file)
target_vocab = np.load("english_words_without.npy")
matrix_len = target_vocab.shape[0]
weights_matrix = np.zeros((matrix_len + 2, 300))
words_found = 0
for i in range(matrix_len):
    print(i)
    try:
        weights_matrix[i] = glove[idx[target_vocab[i]]]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))
weights_matrix[matrix_len] = np.random.normal(scale=0.6, size=(300, ))
weights_matrix[matrix_len+1] = np.random.normal(scale=0.6, size=(300, ))
np.save("initial_embeddings_without.npy",weights_matrix)

Streaming output truncated to the last 5000 lines.
4984
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173

In [ ]:
#================================================================================================================================

In [ ]:

%cd drive/My\ Drive/DL_assignments/A4/machine_translation/
 
import torch
import pickle
import numpy as np
from PIL import Image
from torchvision import transforms
import re
import torch.nn as nn
import torch.nn.functional as F
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method1
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import string

np.random.seed(1)

[Errno 2] No such file or directory: 'drive/My Drive/DL_assignments/A4/machine_translation/'
/content/drive/My Drive/DL_assignments/A4/machine_translation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading data

english_words = np.load("english_words_without_punctuation_dev.npy",allow_pickle=True).tolist() # english vocabulary
tamil_words = np.load("tamil_words_without_punctuation_dev.npy",allow_pickle= True).tolist() # tamil vocabulary
initial_embeddings = np.load("initial_embeddings_without_punctuation_dev_100.npy",allow_pickle=True) # initial glove embeddings for english words 
english_sentences_dev = np.load("english_sentences_without_punctuation_test.npy",allow_pickle=True) # English sentences
tamil_sentences_dev = np.load("tamil_sentences_without_punctuation_test.npy",allow_pickle=True) # Tamil sentences

In [ ]:
english_sentences_dev = np.load("english_sentences_without_punctuation_dev.npy",allow_pickle=True)
tamil_sentences_dev = np.load("tamil_sentences_without_punctuation_dev.npy",allow_pickle=True)

In [ ]:
english_sentences_dev = np.load("english_sentences_shuffled_dev.npy",allow_pickle=True)
tamil_sentences_dev = np.load("tamil_sentences_shuffled_dev.npy",allow_pickle=True)

In [ ]:
# RNN Encoder Decoder
 
class Network(nn.Module):
    def create_emb_layer(self,weights_matrix,flag):
        num_embeddings, embedding_dim = weights_matrix.size()
        if flag == True:
            emb_layer = nn.Embedding(num_embeddings, embedding_dim)
            emb_layer.load_state_dict({'weight': weights_matrix})
        else:
            emb_layer = nn.Embedding(len(tamil_words), embedding_dim)
        emb_layer.weight.requires_grad = True
        return emb_layer
        
    def __init__(self):
        super().__init__()
        
        self.hidden_size = 300
        self.emb_size = 100
        self.output_nodes = 300
        self.num_layers = 1
        self.en_embeddings = self.create_emb_layer(torch.from_numpy(initial_embeddings),True)
        self.ta_embeddings = self.create_emb_layer(torch.from_numpy(initial_embeddings),False)
        self.encoder_rnn = nn.LSTM(self.emb_size, self.hidden_size, self.num_layers, batch_first=True)
        
        self.decoder_rnn = nn.LSTM(self.emb_size + self.output_nodes, self.hidden_size, self.num_layers, batch_first=True)
 
        self.linear_layer = nn.Sequential(
            nn.Linear(self.hidden_size,self.output_nodes)
        )

        self.output_layer = nn.Sequential(
            nn.Linear(self.output_nodes,len(tamil_words))
        )
 
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_size + self.output_nodes, 1)
        )
 
    def forward(self, x,y):
        
        '''
        x_out,(y_out,s_out) = self.encoder_rnn(torch.unsqueeze(self.en_embeddings(x),0))
        en_out = self.linear_layer(x_out)
        output = []
        for y_ in y:
          weights = []
          for x_ in en_out[0]:
            temp = torch.cat((torch.unsqueeze(x_,0),torch.squeeze(self.linear_layer(y_out),0)),axis=1)
            weights.append(self.attention(temp))
          normalized_weights = F.softmax(torch.cat(weights, 1), 1)
          attn_applied = torch.bmm(normalized_weights.unsqueeze(1),en_out.view(1, -1, self.output_nodes))
          input_lstm = torch.cat((attn_applied[0], torch.unsqueeze(self.ta_embeddings(y_),0)), dim = 1)
          x_out,(y_out,s_out) = self.decoder_rnn(input_lstm.unsqueeze(0),(y_out,s_out))
          output.append(self.output_layer(self.linear_layer(y_out)).squeeze(0).squeeze(0))        
        return output

        '''
        out = []
        out_i = []
        first = True
        while True:
            if len(out) == 40:
              return out,out_i 
            if not first:  
                o = self.linear_layer(y_out)
                o = self.output_layer(o)
                o = torch.squeeze(o,0)
                v,i = torch.max(o,1)
                i = i.item()
                if i == len(tamil_words) - 1:
                    break
                elif i == len(tamil_words) - 2:
                    out.append("st")
                    out_i.append(-1)
                else:
                    out.append(tamil_words[i])
                    out_i.append(i)

                weights = []
                for x_ in en_out[0]:
                  temp = torch.cat((torch.unsqueeze(x_,0),torch.squeeze(self.linear_layer(y_out),0)),axis=1)
                  weights.append(self.attention(temp))
                normalized_weights = F.softmax(torch.cat(weights, 1), 1)
                attn_applied = torch.bmm(normalized_weights.unsqueeze(1),en_out.view(1, -1, self.hidden_size))
                input_lstm = torch.cat((attn_applied[0], torch.unsqueeze(self.ta_embeddings(torch.tensor(i)),0)), dim = 1)
                  
                  
            else:
                i = len(tamil_words) - 2
                first = False
                x_out,(y_out,s_out) = self.encoder_rnn(torch.unsqueeze(self.en_embeddings(x),0))
                x_out = self.linear_layer(x_out)
                en_out = x_out
                weights = []
                for x_ in en_out[0]:
                  temp = torch.cat((torch.unsqueeze(x_,0),torch.squeeze(self.linear_layer(y_out),0)),axis=1)
                  weights.append(self.attention(temp))
                normalized_weights = F.softmax(torch.cat(weights, 1), 1)
                attn_applied = torch.bmm(normalized_weights.unsqueeze(1),en_out.view(1, -1, self.hidden_size))
                input_lstm = torch.cat((attn_applied[0], torch.unsqueeze(self.ta_embeddings(torch.tensor(i)),0)), dim = 1)
                
                   

            if i != len(tamil_words) - 1:
                x_out,(y_out,s_out) = self.decoder_rnn(input_lstm.unsqueeze(0),(y_out,s_out))
                
            else:
                break
        return out,out_i
        #'''
        

In [ ]:
print(len(tamil_words))
print(len(english_words))

19338
10298


In [ ]:
e=20001
print("model/model_epoch{}.pth".format(e))


model/model_epoch20001.pth


In [ ]:
print(len(tamil_words))
print(len(english_words))

18750
9983


In [ ]:
# Training 
 
# dev = torch.device("cpu")
 
# if torch.cuda.is_available():  
#   dev = "cuda:0" 
# else:  7
#   dev = "cpu" 
# print("using {}".format(dev))

dev = "cuda:0"
print("using {}".format(dev))
 
model = Network().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)
 
train_from_saved_model = True
start_epoch = 30
num_epochs = 100

if train_from_saved_model:
    checkpoint = torch.load("model/model_machine_trans_attention_lstm_without_emb100_{}.pth".format(start_epoch))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #optimizer = torch.optim.Adam(params=model.parameters(),lr=0.1)
 
print(optimizer.param_groups[0]['lr'])
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01)
# print(optimizer.param_groups[0]['lr'])
'''
for param in model.parameters():
  param.requires_grad = True
'''


for e in range(start_epoch + 1,start_epoch + num_epochs + 1):
    print(e)
    epoch_loss = 0
    optimizer.zero_grad()
    batch_size = 1
    d = 0
    for it in range(int(25500/batch_size)):
        loss = 0
        optimizer.zero_grad()
        expected = english_sentences[it]
        expected = [len(english_words) - 2] + [english_words.index(s.lower()) for s in expected]+ [len(english_words) - 1]
        expected_in = np.array(expected[:-1])
        expected = tamil_sentences[it]
        expected = [len(tamil_words) - 2] + [tamil_words.index(s.lower()) for s in expected]+ [len(tamil_words) - 1]
        expected_out = np.array(expected[1:])
        expected_in_ta  = np.array(expected[:-1])
        output = model(torch.from_numpy(expected_in).to(dev),torch.from_numpy(expected_in_ta).to(dev)) 
        output = torch.stack(output)
        et = torch.from_numpy(expected_out).to(dev)
        loss += criterion(output,et)
        epoch_loss += loss
        if d % 1000 == 0:
          print(d,epoch_loss/d)
        loss.backward()
        optimizer.step()
        d += 1
    print(epoch_loss/25500)
    print(e)
    if e % 5 == 0:
        torch.save({
                    'epoch': e,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': epoch_loss,
                    }, "model/model_machine_trans_attention_lstm_without_emb100_{}.pth".format(e))
        print("saved")

using cuda:0
0.01
31
0 tensor(inf, device='cuda:0', grad_fn=<DivBackward0>)
1000 tensor(0.1866, device='cuda:0', grad_fn=<DivBackward0>)
2000 tensor(0.1842, device='cuda:0', grad_fn=<DivBackward0>)
3000 tensor(0.1857, device='cuda:0', grad_fn=<DivBackward0>)
4000 tensor(0.1878, device='cuda:0', grad_fn=<DivBackward0>)
5000 tensor(0.1828, device='cuda:0', grad_fn=<DivBackward0>)
6000 tensor(0.1855, device='cuda:0', grad_fn=<DivBackward0>)
7000 tensor(0.1785, device='cuda:0', grad_fn=<DivBackward0>)
8000 tensor(0.1766, device='cuda:0', grad_fn=<DivBackward0>)
9000 tensor(0.1754, device='cuda:0', grad_fn=<DivBackward0>)
10000 tensor(0.1727, device='cuda:0', grad_fn=<DivBackward0>)
11000 tensor(0.1708, device='cuda:0', grad_fn=<DivBackward0>)
12000 tensor(0.1711, device='cuda:0', grad_fn=<DivBackward0>)
13000 tensor(0.1683, device='cuda:0', grad_fn=<DivBackward0>)
14000 tensor(0.1662, device='cuda:0', grad_fn=<DivBackward0>)
15000 tensor(0.1659, device='cuda:0', grad_fn=<DivBackward0>)
160

In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
            }, "model/model_epoch_final_131.pth")
print("saved")

saved


In [ ]:
# Testing 

dev = torch.device("cpu")

model = Network().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)
 
checkpoint = torch.load("model_machine_trans_attention_lstm_without_emb100_45.pth",map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
 
model.eval()
tot_score_1 = 0
tot_score_2 = 0
tot_score_3 = 0
tot_score_4 = 0
start = 00
end = 1000
div1 = 0
div2 = 0
div3 = 0
div4 = 0
div = 0
refs = []
candidates = []
for d in range(start,end):
    print(d)
    expected = english_sentences_dev[d]
    # print(expected)
    e = [len(english_words) - 2]
    for s in expected:
      if s.lower() in english_words:
        e.append(english_words.index(s.lower()))
      else:
        e.append(len(english_words) - 2)
    e.append(len(english_words) - 1)
    # expected = [len(english_words) - 2] + [english_words.index(s.lower()) for s in expected]+ [len(english_words) - 1]
    # expected = [-1,-1]
    expected_in = np.array(e[:-1])
    
    expected = tamil_sentences_dev[d]
    print(expected)
    # e = expected
    # expected = [len(tamil_words) - 2] + [tamil_words.index(s.lower()) for s in expected]+ [len(tamil_words) - 1]
    # e = [len(tamil_words) - 2,]
    # e = [len(tamil_words) - 2]
    # for s in expected:
    #   if s.lower in tamil_words:
    #     e.append(tamil_words.index(s.lower()))
    #   else:
    #     e.append(len(tamil_words) - 2)

    # expected_out = np.array(expected[1:])
    # exp = expected_out[:-1]
    # print(expected_out)
    expected_out = np.array([0,0])
    output,out_i = model(torch.from_numpy(expected_in).to(dev),torch.from_numpy(expected_out).to(dev))        
    print(output)
    print()
    # input()
    # refs.append([e])
    # candidates.append(output)

    # if len(out_i) >= 1:
    #   tot_score_1 += bleu_score([output], [[e]],weights=(1, 0, 0, 0))
    #   div1 += 1
    # if len(out_i) >= 2:  
    #   tot_score_2 += bleu_score([output], [[e]],weights=(0.5, 0.5, 0, 0))
    #   div2 += 1
    # if len(out_i) >= 3:   
    #   tot_score_3 += bleu_score([output], [[e]],weights=(0.33, 0.33, 0.33, 0))
    #   div3 += 1
    # if len(out_i) >= 4:
    #   tot_score_4 += bleu_score([output], [[e]],weights=(0.25, 0.25, 0.25, 0.25))
    #   div4 += 1

    d += 1
    if len(out_i) >= 4:
        tot_score_1 += sentence_bleu([expected], output,weights=(1, 0, 0, 0),smoothing_function=smoothie)
        tot_score_2 += sentence_bleu([expected], output,weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie)
        tot_score_3 += sentence_bleu([expected], output,weights=(0.33, 0.33, 0.33, 0),smoothing_function=smoothie)
        tot_score_4 += sentence_bleu([expected], output,weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie)
        div += 1

# tot_score_1 += corpus_bleu(refs, candidates,weights=(1, 0, 0, 0), smoothing_function=smoothie)
# tot_score_2 += corpus_bleu(refs, candidates,weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
# tot_score_3 += corpus_bleu(refs, candidates,weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
# tot_score_4 += corpus_bleu(refs, candidates,weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)

# print(tot_score_1/div1)
# print(tot_score_2/div2)
# print(tot_score_3/div3)
# print(tot_score_4/div4)
# print(div1)
# print(div2)
# print(div3)
# print(div4)

print(div)
print(tot_score_1/div)
print(tot_score_2/div)
print(tot_score_3/div)
print(tot_score_4/div)

0
['SIS', 'மூன்று', '36', 'வயது', 'நடுத்தர', 'ஊதியம்', 'கடந்த', 'மூன்று', 'ஆண்டுகளில்']
['மூன்று', 'நாட்கள்', 'ரன்', 'வெற்றி']

1
['நீங்கள்', 'தேர்வு', 'செய்ய', 'வேண்டும்', '']
['நீங்கள்', 'ஒரு', 'தேர்வு', 'வேண்டும்']

2
['பிட்ச்', 'நீங்கள்', 'நல்ல', 'உங்கள்', 'என்', 'என்', 'அலுவலக', 'outta']
['என்', 'அலுவலகத்தில்', 'நீங்கள்', 'இந்த', 'மதிய', 'உங்கள்', 'சிறந்த', 'திரும்ப', 'இருக்கிறதா']

3
['மற்றும்', 'உலக', 'எங்களை', 'பற்றி', 'மறக்க', 'முடியாது']
['அமெரிக்க', 'நம்', 'மக்கள்', 'பற்றி']

4
['என்', 'செல்', 'போன்', 'எச்சரிக்கைகள்', 'மூலம்']
['தொலைபேசியில்', 'என்', 'தொலைபேசியில்']

5
['இந்த', 'சாரணர்', 'கப்பல்', 'இருந்தது', 'வெற்றிடத்தை', 'செலுத்தப்படுகின்றன', 'ஆயிரக்கணக்கான', 'ஒன்றாகும்']
['இந்த', 'ஒரு', 'போர்வீரன்', 'ஒரு', 'காற்று', 'கட்டி', 'உங்கள்', 'மீது', 'தந்தை', 'தான்']

6
['பெண்', 'பாடல்உள்ளதுஒரு', 'இருண்ட', 'மற்றும்', 'ஒரு', 'நான்', 'வாழ்க்கை', 'பதற்றமான', 'பக்க']
['சிங்கர்', 'என்ற', 'கருப்பு', 'கிரகத்தில்', 'ஒரு', 'சிறிய', 'துளை', 'என்', 'இருக்கும்']

7
['நேரங்களிலேயே', 'அவர

In [ ]:
# final training - emb100_55 
0.947673009826474
0.9418868578116316
0.8513988346872535
0.743278313717439

#on valid - 
0.2773284027605993
0.16507948405180653
0.1310028667843173
0.1146134980385189

# on test
0.1612009809761761
0.05673242932937537
0.02957693076697403
0.020315791860848274

In [ ]:
final viva 
0.17733835834046052
0.07376036533150933
0.04831206133699982
0.03933365574787462